# **ECSE 415 Final Project**
---
**Remi Carriere**     260636007

**Gohar Saqib Fazal** 260840565

In [ ]:
# import libraries
!pip install ultralytics -q
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from ultralytics import YOLO
import IPython
from IPython.display import Video, display

## **Video Preprocessing**
---

In [ ]:
# Splitting the video into multiple parts of smaller frame sizes to speed testing
def split_video(input_path, output_path1, output_path2):
    input_path = './data/mcgill_drive.mp4'
    output_path1 = './data/mcgill_drive_part1.mp4'
    output_path2 = './data/mcgill_drive_part2.mp4'

    # Open the video file
    cap = cv2.VideoCapture(input_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the midpoint to split the video into two equal parts
    midpoint = frame_count // 4

    # Create VideoWriter objects for the two output videos
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out1 = cv2.VideoWriter(output_path1, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))
    out2 = cv2.VideoWriter(output_path2, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

    # Process each frame and write to the respective output video
    for i in range(frame_count):
        ret, frame = cap.read()

        if not ret:
            break

        if i < midpoint:
            out1.write(frame)
        else:
            out2.write(frame)

    # Release VideoWriter objects and close the video file
    out1.release()
    out2.release()
    cap.release()

## **Loading the Model**
---
The YOLOv8 pretrained models have been trained on the COCO dataset; however, they lack separate classes for pedestrians and cyclists. In an effort to address this limitation, we attempted to train our own model using a dataset that includes classes for cyclists and pedestrians (refer to https://universe.roboflow.com/adp-l8hde/yolov8-6apfg/dataset/4). Unfortunately, despite multiple attempts, our efforts were unsuccessful. We initially trained the YOLOv8 nano model for 200 epochs, but it did not have satisfactory performance in reliably tracking cars, pedestrians, and cyclists. Subsequently, we attempted to train the YOLOv8 medium model. Regrettably, we encountered a constraint in the form of limited access to a sufficiently powerful GPU, preventing us from training the model within a reasonable timeframe.

In [ ]:
model = YOLO('yolov8m.pt')
results = model.track(source='./data/mcgill_drive.mp4', show=True, tracker="botsort.yaml", save=True, show_conf=False, conf=0.45, persist=True)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/1071) /notebooks/data/mcgill_drive.mp4: 384x640 4 cars, 31.9ms
video 1/1 (2/1071) /notebooks/data/mcgill_drive.mp4: 384x640 4 cars, 14.9ms
video 1/1 (3/1071) /notebooks/data/mcgill_drive.mp4: 384x640 4 cars, 14.0ms
video 1/1 (4/1071) /notebooks/data/mcgill_drive.mp4: 384x640 4 cars, 18.2ms
video 1/1 (5/1071) /notebooks/data/mcgill_drive.mp4: 384x640 4 cars, 16.5ms
vi

In [ ]:
model = YOLO('yolov8m.pt')
results_2 = model.track(source='./data/st-catherines_drive.mp4', show=True, tracker="botsort.yaml", save=True, show_conf=False, conf=0.45, persist=True)

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 118.1 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 2.1s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()





WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/1477) /notebooks/data/st-catherines_drive.mp4: 384x640 3 persons, 6 cars, 1 truck, 14.8ms
video 1/1 (2/1477) /notebooks/data/st-catherines_drive.mp4: 384x640 3 persons, 6 cars, 1 truck, 9.0ms
video 1/1 (3/1477) /notebooks/data/st-catherines_drive.mp4: 384x640 3 persons, 6 cars, 1 truck, 8.4ms
video 1/1 (4/1477) /notebooks/data/st-catherines_drive.mp4: 384x640 3 persons, 6 cars, 1 truck, 8.5ms
video 1/1 (5/1477) /notebooks/data/st-catherine

# **Analytics**
---

## **Number of parked cars passed and Number of moving cars passed**
---

### **Detailed description of the overall approach taken. State clearly any assumptions that you made.**
---
1.   ***Approach:***     

> YOLOv8 is used for detecting and tracking cars in each frame of the dashcam video. The class number for cars is 2 and can be found using `results[id].names`. The script iterates through each frame, extracting information about detected cars, such as their IDs and bounding boxes. It maintains a dictionary (`carsDict`) to store information about each car, including its average displacement, whether it has passed, whether it is parked, and a history of its presence in previous frames.

> `seenPrev` is a property in `carsDict` dictionary. It is a temporal context window that spans the last 15 frames (half a second in a 30 frames per second video). It acts as a history tracker, indicating whether a specific car has been consistently visible in recent frames. The primary purpose of seenPrev is to identify if a car has passed the field of view of the dashcam. If a car is not detected in any of the last 15 frames, it is considered as "passed." In a scenario where cars can be stationary (e.g., at traffic lights, stop signs, or during turns), using `seenPrev` helps dynamically analyze the motion behavior over a short time span. After updating `seenPrev` based on the current frame's detections, the code calculates the displacement (norm) for each detected car.

> The `norm_calc` function calculates the displacement of each detected car between consecutive frames. The displacement is based on the Euclidean distance between the centroids of the bounding boxes.

> The code classifies cars as parked or moving based on their average displacement over time. If the average displacement falls within specified thresholds (`min_threshold` and `max_threshold`), a car is classified as parked.

> The script counts the number of parked and moving cars based on their classification and whether they have passed. The final results are printed, indicating the number of parked cars and moving cars passed in the dashcam video.

2.   ***Assumptions:***

> Displacement between consecutive frames is calculated using the Euclidean distance between the centroids of bounding boxes around detected cars. This assumes that the centroid of a car remains a representative point for its position.

> The code uses threshold values for average displacement to classify cars as parked or moving. These thresholds (`min_threshold` and `max_threshold`) are assumed to be appropriate for the characteristics of the given video. Adjustments may be needed based on specific video conditions.


In [ ]:
def norm_calc(frame, index, id):
    # Initialize displacement variable
    displacement = 0

    # results[frame_index].boxes.cls contains the class numbers
    car_class_number = 2  # Replace with the actual class number for cars

    # Get indices where the detected class is the car class in the previous and current frames
    car_indices_1 = np.where(results[index - 1].boxes.cls == car_class_number)
    car_indices_2 = np.where(results[index].boxes.cls == car_class_number)

    # Extract bounding boxes and IDs for the detected cars in the previous frame
    for car_index_1 in car_indices_1:
        bbox1 = results[index - 1].boxes.xyxy[car_index_1, :4]
        car_id_1 = results[index - 1].boxes.id[car_index_1]

    # Extract bounding boxes and IDs for the detected cars in the current frame
    for car_index_2 in car_indices_2:
        bbox2 = results[index].boxes.xyxy[car_index_2, :4]
        car_id_2 = results[index].boxes.id[car_index_2]

    # Find matching elements (cars with the same ID) between the two frames
    matching_elements = car_id_1[torch.isin(car_id_1, car_id_2)]

    # Iterate through matching elements to calculate displacement
    for car_id in matching_elements:
        # Find the indices of the matching elements in each frame
        car_index_1 = np.where(results[index - 1].boxes.id == car_id)[0][0]
        car_index_2 = np.where(results[index].boxes.id == car_id)[0][0]

        # Get the bounding boxes for the matching cars
        bbox1 = results[index - 1].boxes.xyxy[car_index_1, :4].detach().cpu().numpy()
        bbox2 = results[index].boxes.xyxy[car_index_2, :4].detach().cpu().numpy()

        # Convert bounding boxes to integers
        bbox1 = bbox1.astype(int)
        bbox2 = bbox2.astype(int)

        # Check if the current car ID matches the specified ID
        if torch.all(car_id == id):
            # Calculate the displacement using Euclidean distance between centroids of bounding boxes
            displacement = np.linalg.norm(np.array(bbox2[:2]) - np.array(bbox1[:2]))

    # Return the calculated displacement
    return displacement


In [ ]:
def norm_calc(frame, index, id):
    # Initialize displacement variable
    displacement = 0

    # results[frame_index].boxes.cls contains the class numbers
    car_class_number = 2  # Replace with the actual class number for cars

    # Get indices where the detected class is the car class in the previous and current frames
    car_indices_1 = np.where(results_2[index - 1].boxes.cls == car_class_number)
    car_indices_2 = np.where(results_2[index].boxes.cls == car_class_number)

    # Extract bounding boxes and IDs for the detected cars in the previous frame
    for car_index_1 in car_indices_1:
        bbox1 = results_2[index - 1].boxes.xyxy[car_index_1, :4]
        car_id_1 = results_2[index - 1].boxes.id[car_index_1]

    # Extract bounding boxes and IDs for the detected cars in the current frame
    for car_index_2 in car_indices_2:
        bbox2 = results_2[index].boxes.xyxy[car_index_2, :4]
        car_id_2 = results_2[index].boxes.id[car_index_2]

    # Find matching elements (cars with the same ID) between the two frames
    matching_elements = car_id_1[torch.isin(car_id_1, car_id_2)]

    # Iterate through matching elements to calculate displacement
    for car_id in matching_elements:
        # Find the indices of the matching elements in each frame
        car_index_1 = np.where(results_2[index - 1].boxes.id == car_id)[0][0]
        car_index_2 = np.where(results_2[index].boxes.id == car_id)[0][0]

        # Get the bounding boxes for the matching cars
        bbox1 = results_2[index - 1].boxes.xyxy[car_index_1, :4].detach().cpu().numpy()
        bbox2 = results_2[index].boxes.xyxy[car_index_2, :4].detach().cpu().numpy()

        # Convert bounding boxes to integers
        bbox1 = bbox1.astype(int)
        bbox2 = bbox2.astype(int)

        # Check if the current car ID matches the specified ID
        if torch.all(car_id == id):
            # Calculate the displacement using Euclidean distance between centroids of bounding boxes
            displacement = np.linalg.norm(np.array(bbox2[:2]) - np.array(bbox1[:2]))

    # Return the calculated displacement
    return displacement


In [ ]:
# Initialize a dictionary to store information about cars
# Format: dict = {id: {seenPrev, avg_norm, numerOfFramesProcessed, Parked, isPassed, final_avg_norm}}
carsDict = {}

# Initialize a variable to keep track of the number of frames processed
numberOfFramesProcessed = 0

# Path to the input video file
input_path = "./data/st-catherines_drive.mp4"  # Change to the path of avi file generated by model tracker

# Open the video file for reading
cap = cv2.VideoCapture(input_path)

# Get the frames per second and total frame count of the video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Process each frame of the video
for i in range(frame_count):
    # Read the next frame from the video
    ret, frame = cap.read()

    # Break the loop if there are no more frames
    if not ret:
        break

    # Initialize a list to store IDs of detected cars in the current frame
    id_list = []

    # Class number for cars in the YOLOv8 model
    car_class_number = 2

    # Get indices where the detected class is the car class
    car_indices_1 = np.where(results_2[i].boxes.cls == car_class_number)

    # Extract IDs of detected cars in the current frame
    for car_index_1 in car_indices_1:
        car_id_1 = results_2[i].boxes.id[car_index_1]

    # Iterate through detected car IDs in the current frame
    for id in car_id_1:
        id = int(id)
        id_list.append(id)

        # Check if the ID is not in the dictionary, initialize properties
        if id not in carsDict.keys():
            carsDict[id] = {}
            carsDict[id]['seenPrev'] = [0] * 14 + [1]  # Initialize seenPrev list with 14 zeros and 1
            carsDict[id]['avg_norm'] = []  # Initialize list to store displacement values
            carsDict[id]['numerOfFramesProcessed'] = 1
            carsDict[id]['Parked'] = False
            carsDict[id]['isPassed'] = False

        else:
            # Update seenPrev list and set isPassed to False
            carsDict[id]['seenPrev'] = carsDict[id]['seenPrev'][1:] + [1]
            carsDict[id]['isPassed'] = False

            # Calculate displacement using norm_calc function and update properties
            displacement = norm_calc(frame, i, id)
            carsDict[id]['avg_norm'].append(displacement)
            carsDict[id]['numerOfFramesProcessed'] += 1

    # Update seenPrev list for IDs not present in the current frame
    for i in carsDict.keys():
        if all(element == 0 for element in carsDict[i]['seenPrev']):
            carsDict[i]['isPassed'] = True
        if i not in id_list:
            carsDict[i]['seenPrev'] = carsDict[i]['seenPrev'][1:] + [0]

# Release the video capture object
cap.release()

# Set thresholds for determining parked and moving cars
min_threshold = 6
max_threshold = 25

# Initialize counters for parked and moving cars
parkedCars = 0
movingCars = 0

# Process the information for each car in the dictionary
for id in carsDict.keys():
    # Calculate the final average displacement for each car
    carsDict[id]['final_avg_norm'] = np.mean(carsDict[id]['avg_norm'])

    # Check if the final average displacement falls within the specified thresholds
    if min_threshold < carsDict[id]['final_avg_norm'] < max_threshold:
        carsDict[id]['Parked'] = True

    # Check conditions for parked and moving cars
    if carsDict[id].get('isPassed', False) and carsDict[id]['Parked']:
        parkedCars += 1
    if carsDict[id].get('isPassed', False) and not carsDict[id]['Parked']:
        movingCars += 1

# Print the final results
print("Number of Parked Cars Passed - St-Catherines Drive:", parkedCars)
print("Number of Moving Cars Passed - St-Catherines Drive:", movingCars)

Number of Parked Cars Passed - St-Catherines Drive: 52
Number of Moving Cars Passed - St-Catherines Drive: 20


In [ ]:
# Initialize a dictionary to store information about cars
# Format: dict = {id: {seenPrev, avg_norm, numerOfFramesProcessed, Parked, isPassed, final_avg_norm}}
carsDict = {}

# Initialize a variable to keep track of the number of frames processed
numberOfFramesProcessed = 0

# Path to the input video file
input_path = "./data/mcgill_drive.mp4"  # Change to the path of avi file generated by model tracker

# Open the video file for reading
cap = cv2.VideoCapture(input_path)

# Get the frames per second and total frame count of the video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Process each frame of the video
for i in range(frame_count):
    # Read the next frame from the video
    ret, frame = cap.read()

    # Break the loop if there are no more frames
    if not ret:
        break

    # Initialize a list to store IDs of detected cars in the current frame
    id_list = []

    # Class number for cars in the YOLOv8 model
    car_class_number = 2

    # Get indices where the detected class is the car class
    car_indices_1 = np.where(results[i].boxes.cls == car_class_number)

    # Extract IDs of detected cars in the current frame
    for car_index_1 in car_indices_1:
        car_id_1 = results[i].boxes.id[car_index_1]

    # Iterate through detected car IDs in the current frame
    for id in car_id_1:
        id = int(id)
        id_list.append(id)

        # Check if the ID is not in the dictionary, initialize properties
        if id not in carsDict.keys():
            carsDict[id] = {}
            carsDict[id]['seenPrev'] = [0] * 14 + [1]  # Initialize seenPrev list with 14 zeros and 1
            carsDict[id]['avg_norm'] = []  # Initialize list to store displacement values
            carsDict[id]['numerOfFramesProcessed'] = 1
            carsDict[id]['Parked'] = False
            carsDict[id]['isPassed'] = False

        else:
            # Update seenPrev list and set isPassed to False
            carsDict[id]['seenPrev'] = carsDict[id]['seenPrev'][1:] + [1]
            carsDict[id]['isPassed'] = False

            # Calculate displacement using norm_calc function and update properties
            displacement = norm_calc(frame, i, id)
            carsDict[id]['avg_norm'].append(displacement)
            carsDict[id]['numerOfFramesProcessed'] += 1

    # Update seenPrev list for IDs not present in the current frame
    for i in carsDict.keys():
        if all(element == 0 for element in carsDict[i]['seenPrev']):
            carsDict[i]['isPassed'] = True
        if i not in id_list:
            carsDict[i]['seenPrev'] = carsDict[i]['seenPrev'][1:] + [0]

# Release the video capture object
cap.release()

# Set thresholds for determining parked and moving cars
min_threshold = 3.5
max_threshold = 7

# Initialize counters for parked and moving cars
parkedCars = 0
movingCars = 0

# Process the information for each car in the dictionary
for id in carsDict.keys():
    # Calculate the final average displacement for each car
    carsDict[id]['final_avg_norm'] = np.mean(carsDict[id]['avg_norm'])

    # Check if the final average displacement falls within the specified thresholds
    if min_threshold < carsDict[id]['final_avg_norm'] < max_threshold:
        carsDict[id]['Parked'] = True

    # Check conditions for parked and moving cars
    if carsDict[id].get('isPassed', False) and carsDict[id]['Parked']:
        parkedCars += 1
    if carsDict[id].get('isPassed', False) and not carsDict[id]['Parked']:
        movingCars += 1

# Print the final results
print("Number of Parked Cars Passed - McGill Drive:", parkedCars)
print("Number of Moving Cars Passed - McGill Drive:", movingCars)

Number of Parked Cars Passed - McGill Drive: 8
Number of Moving Cars Passed - McGill Drive: 47


### **Summary of program output on the two videos, with comparison to manually obtained ground truth values**
---

1.   ***McGill Drive Video:***

**Number of Moving Cars Passed:**
*   Program Output: 47
*   Ground Truth: 30
*   Comparison: The program slightly overestimated the number of moving cars.

**Number of Parked Cars Passed:**
*   Program Output: 8
*   Ground Truth: 14
*   Comparison: The program slightly underestimated the number of parked cars. This was due to a low max_threshold set in the code.

**Observations:**
The program generally performed well but showed a tendency to overestimate moving cars and slightly underestimate parked cars.

2.   ***St-Catherine Drive Video:***

**Number of Moving Cars Passed:**
*   Program Output: 20
*   Ground Truth: 5
*   Comparison: The program slightly overestimated the number of moving cars.

**Number of Parked Cars Passed:**
*   Program Output: 52
*   Ground Truth: 56
*   Comparison: The program slightly underestimated the number of parked cars.

**Observations:**
Similar to the McGill Drive video, the program demonstrated a tendency to overestimate moving cars and underestimate parked cars.



### **Discussion of program performance and problems**
---
1. ***Object Detection Model Inaccuracy:***

> The YOLO tracking model occasionally misidentifies reflections as actual cars. This can lead to discrepancies in the program's analysis, especially in scenarios where reflections are present.

2. ***Inconsistencies in Object Tracking:***

> Tracking inconsistencies occur when a car is initially identified with a specific ID, but a subsequent object (e.g., a truck) passes in front of it. Now, the object tracker assigns a new ID to the same car once the subsequent object passed The program may inaccurately count multiple IDs for a single physical object, affecting the accuracy of passed and moving car calculations.

3. ***Undetected Cars:***

> Towards the end of the video, certain cars are undetected because the algorithm employs a 15-frame waiting period before classifying a car as "passed" or "not passed." This waiting period introduces a delay in the detection of cars, potentially resulting in missed detections, especially in scenarios where cars momentarily leave the frame.

4. Norm Calculation Challenges:

> The McGill Drive video's dynamic nature, coupled with the absence of a consistent stationary reference point on the two-lane road, poses challenges for accurate norm calculations. The changing field of view and moving camera contribute to inaccuracies in determining whether detected cars are in motion or stationary. While the implemented norm thresholds perform reasonably well, addressing these challenges may require refining the norm calculation methodology for improved precision in distinguishing between moving and stationary vehicles.

## **Number of pedestrians passed**
---

### **Detailed description of the overall approach taken. State clearly any assumptions that you made.**
---
1. **Approach:**

> We opted to utilize the YOLOv8 large model and devised a method to count pedestrians by checking the overlap of their bounding boxes with those of bicycles. While this approach may fail when a pedestrian is in close proximity to a bicycle, it yielded better overall performance. This was due to the pretrained model's enhanced reliability in tracking cars and persons.

2. **Assumptions:**

> It was assumed that pedestrians will never walk in front of cyclicts. Otherwise, their bounding box will overlap with that of the cyclist causing inaccuracies in the prediction

> Cyclist and person bounding boxes should be detected simultaneously for this approach to work.


In [ ]:
def intersecting_boxes(box1, box2):
    # Extract coordinates from boxes
    x1, y1, x2, y2 = box1[0]
    x3, y3, x4, y4 = box2[0]

    # Check for intersection
    if x2 < x3 or x4 < x1 or y2 < y3 or y4 < y1:
        # No intersection
        return False
    else:
        # Intersection
        return True

labels = ['person', 'bicycle', 'car']


def count_pedestians(results):
    pedestrian_count = 0
    tracked_pedestrian_IDs = []
    # Count pedestrians
    for result in results:
        pedestrian_boxes = []
        bicycle_boxes = []
        for box in result.boxes:
            if not box.id: continue
            # if we find an untracked person
            if int(box.cls) == 0 and not int(box.id) in tracked_pedestrian_IDs:
                pedestrian_boxes.append(box.xyxyn) # save the bouding box
                tracked_pedestrian_IDs.append(int(box.id)) # The person is now tracked
            # if we find a bicycle
            elif int(box.cls) == 1:
                bicycle_boxes.append(box.xyxyn) # save the bouding box

        # Now, we check ig the person bounding box intersects a bicycle box
        # If they dont intersect, we assume the person is a pedestrian
        for pedestrian_box in pedestrian_boxes:
            if not bicycle_boxes:
                pedestrian_count+=1
            else:
                for bicycle_box in bicycle_boxes:
                    if not intersecting_boxes(bicycle_box, pedestrian_box):
                        pedestrian_count+=1
    return pedestrian_count

In [ ]:
pedestrian_count_mcgill_drive = count_pedestians(results)

print("Number of Pedestrians Passed - McGill Drive: ", pedestrian_count_mcgill_drive)

Number of Pedestrians Passed - McGill Drive:  47


In [ ]:
pedestrian_count_st_catherines_drive = count_pedestians(results_2)

print("Number of Pedestrians Passed - St-Catherines Drive: ", pedestrian_count_st_catherines_drive)

Number of Pedestrians Passed - St-Catherines Drive:  115


### **Summary of program output on the two videos, with comparison to manually obtained ground truth values**
---
1.   ***McGill Drive Video:***

**Number of Pedestrians Passed:**
*   Program Output: 47
*   Ground Truth: 35
*   Comparison: The program slightly overestimated the number of pedestrians.

**Observations:**
The program generally performed well but showed a tendency to overestimate the number of pedestrians passed.

2.   ***St-Catherine Drive Video:***

**Number of Pedestrians Passed:**
*   Program Output: 115
*   Ground Truth: 112
*   Comparison: The program slightly overestimated the number of pedestrians.

**Observations:**
Similar to the McGill Drive video, the program demonstrated a tendency to overestimate the number of pedestrians passed. It generally, did better for the St-Catherine Drive video because there were less cyclists.

### **Discussion of program performance and problems**
---
> The number of pedestrians are generally overestimated because of the inability of YOLOv8 model to do accurate detection on the cyclists and person simultaneously.

## **Maximum speed in km/hour of the car with the dashcam**
---

### **Detailed description of the overall approach taken. State clearly any assumptions that you made.**
---
1.   ***Alternative approaches considered:***

* **Optical Flow:** Initially considered using optical flow for speed estimation. However, it required a stationary point of reference, which wasn't available in the video.

* **Pre-trained Models:** Attempted to use pre-trained models for optical flow online, but faced technical issues with Keras inconsistencies, hindering implementation.

2. ***Current approach:***

> The code utilizes OpenCV to read the video file, extracting frames and determining key properties like frames per second (fps) and total frame count (`frame_count`). In each frame, the YOLOv8 model is used to detect traffic lights. The class number for traffic lights is 9, and can be confirmed using `results[id].names` (`traffic_light_number`).

> Upon detecting a traffic light, the code calculates the frames between consecutive detections (`frames_between_detections`). If the number of frames between detections exceeds the specified minimum (`min_frames_between_detections`), it updates the final_frames variable and records the current frame as the previous_detection_frame. The speed is calculated using the formula `speed = distance / (final_frames / fps)`. The maximum speed is updated if the calculated speed exceeds the existing maximum speed. The maximum speed is then converted to kilometers per hour (`max_speed_km_h`).

2.   ***Assumptions:***

> This code assumes consistency in the distance between consecutive traffic lights. I utilized Google Maps data to determine average block sizes between traffic lights in metres.

> This code also assumes that the speed remains constant between intersections.

In [ ]:
# Path to the input video file
input_path = "./data/st-catherines_drive.mp4"  # Change to the path of avi file generated by model tracker

# Open the video file for reading
cap = cv2.VideoCapture(input_path)

# Get the frames per second and total frame count of the video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

distance = 55 # Average distance between two traffic light signals
max_speed = 0  # Variable to store the maximum speed
final_frames = 0

# Ensuring that the no two traffic signal detections are
min_frames_between_detections = 100  # Adjust as needed

# Initialize variables
previous_detection_frame = None

# Process each frame of the video
for i in range(frame_count):
    # Read the next frame from the video
    ret, frame = cap.read()

    # Break the loop if there are no more frames
    if not ret:
        break

    # Class number for traffic lights in the YOLOv8 model
    traffic_light_number = 9

    # Get indices where the detected class is the traffic light
    traffic_indices_1 = np.where(results_2[i].boxes.cls == traffic_light_number)

    # Check if there are any detected traffic signals
    if len(traffic_indices_1[0]) > 0:
        # If it's the first detection, record the frame number
        if previous_detection_frame is None:
            previous_detection_frame = i
        else:
            # If it's not the first detection, calculate the frames between detections
            frames_between_detections = i - previous_detection_frame

            # If frames between detections meet the minimum requirement, update final_frames and previous_detection_frame
            if frames_between_detections >= min_frames_between_detections:
                final_frames = frames_between_detections
                previous_detection_frame = i

    # If final_frames has been updated (indicating a valid detection), calculate speed and update max_speed
    if final_frames > 0:
        speed = distance / (final_frames / fps)
        max_speed = max(max_speed, speed)

# Convert max_speed from m/s to km/h
max_speed_km_h = max_speed * 3.6

# Print the maximum speed of the car
print(f'Max Average Speed of the Car with the Dashcam - St-Catherines Drive: {max_speed_km_h} km/h')

# Release the video capture object
cap.release()


Max Average Speed of the Car with the Dashcam - St-Catherines Drive: 41.608695652173914 km/h


In [ ]:
# Path to the input video file
input_path = "./data/mcgill_drive.mp4"  # Change to the path of avi file generated by model tracker

# Open the video file for reading
cap = cv2.VideoCapture(input_path)

# Get the frames per second and total frame count of the video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

distance = 55 # Average distance between two traffic light signals
max_speed = 0  # Variable to store the maximum speed
final_frames = 0

# Ensuring that the no two traffic signal detections are
min_frames_between_detections = 150  # Adjust as needed

# Initialize variables
previous_detection_frame = None

# Process each frame of the video
for i in range(frame_count):
    # Read the next frame from the video
    ret, frame = cap.read()

    # Break the loop if there are no more frames
    if not ret:
        break

    # Class number for traffic lights in the YOLOv8 model
    traffic_light_number = 9

    # Get indices where the detected class is the traffic light
    traffic_indices_1 = np.where(results[i].boxes.cls == traffic_light_number)

    # Check if there are any detected traffic signals
    if len(traffic_indices_1[0]) > 0:
        # If it's the first detection, record the frame number
        if previous_detection_frame is None:
            previous_detection_frame = i
        else:
            # If it's not the first detection, calculate the frames between detections
            frames_between_detections = i - previous_detection_frame

            # If frames between detections meet the minimum requirement, update final_frames and previous_detection_frame
            if frames_between_detections >= min_frames_between_detections:
                final_frames = frames_between_detections
                previous_detection_frame = i

    # If final_frames has been updated (indicating a valid detection), calculate speed and update max_speed
    if final_frames > 0:
        speed = distance / (final_frames / fps)
        max_speed = max(max_speed, speed)

# Convert max_speed from m/s to km/h
max_speed_km_h = max_speed * 3.6

# Print the maximum speed of the car
print(f'Max Average Speed of the Car with the Dashcam - McGill Drive: {max_speed_km_h} km/h')

# Release the video capture object
cap.release()


Max Average Speed of the Car with the Dashcam - McGill Drive: 34.59036144578313 km/h


### **Summary of program output on the two videos, with comparison to manually obtained ground truth values**
---
1. ***McGill Drive Video:***
**Max Speed of Car with Dashcam:** 34.5 km/h

**Observations:** The algorithm effectively calculated the maximum speed of the car with the dashcam on McGill Drive. Due to inaccuracies in traffic light detection and inconsistent norm calculations, some discrepancies exist.

2. ***St-Catherine Drive Video:***
**Max Speed of Car with Dashcam:** 41.6 km/h

**Observations:** The algorithm successfully calculated the maximum speed of the car with the dashcam on St-Catherine Drive with a decent accuracy. Due to inaccuracies in traffic light detection and inconsistent norm calculations, some discrepancies exist.


### **Discussion of program performance and problems**
---
1. ***Inaccuracies in Object Detection:***

> The accuracy of the program heavily depends on the object detection model. Inaccuracies in identifying traffic lights, cars, and other elements can lead to discrepancies in speed calculation.

2. ***Assumptions About Speed Consistency:***

>The program assumes a consistent speed between intersections. In real-world scenarios, vehicle speed may vary, especially in urban environments with traffic, stops, and accelerations.